In [1]:
import os
import json
import tiktoken
from collections import Counter
import random
from datasets import Dataset, IterableDataset, load_from_disk
import pickle
import numpy as np
from phonenumbers import PhoneNumberMatcher
import multiprocess as mp
from collections import defaultdict
import nltk
import matplotlib.pyplot as plt
import numpy as np
import sys

sys.path.append('../scripts')
from utils import *
import detect_en
import mask_pii
import gopher_filter

In [1]:
ds_name = 'big-5p'
ds_path = '../data/'+ds_name

In [2]:
import sys
sys.path.append('../')

import clean_corpus  # Import the clean_corpus package
dataset = clean_corpus.load_webz_dataset(ds_name)  # Load the dataset using the function

# Display some information about the loaded dataset
print(f"Loaded dataset: {ds_name}")
print(f"Number of rows: {len(dataset)}")
print(f"Features: {dataset.features}")

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [2]:
2**20+1

1048577

In [22]:
import numpy as np
for d in range(28):
    x=np.random.normal(0, 1, 2**d)
    y=np.random.normal(0, 1, 2**d)
    m=np.absolute((x*y)).mean()
    print(d,2**d,m)

0 1 1.3814861993795258
1 2 1.4807161803740103
2 4 1.2638512605567982
3 8 0.2058977094545617
4 16 0.793480021811575
5 32 0.8836150829883063
6 64 0.793209431476516
7 128 0.6758602041011978
8 256 0.771187107911777
9 512 0.6637006609256471
10 1024 0.6388875338494893
11 2048 0.6310464773338049
12 4096 0.6168976865518092
13 8192 0.6225843255781469
14 16384 0.6360176222606685
15 32768 0.6296328563037465
16 65536 0.6370584854917849
17 131072 0.6343303249709334
18 262144 0.6373374175702228
19 524288 0.6371410419837418
20 1048576 0.6375264413644244
21 2097152 0.6364633804144381
22 4194304 0.636754711018467
23 8388608 0.6365915541482641
24 16777216 0.6365119388001268
25 33554432 0.6366154405499375
26 67108864 0.6364970505904106
27 134217728 0.6366934333549402


In [ ]:
curl = {}
for ds_name in DSS.keys():
    curl[ds_name] = Counter(site(x) for x in load_ds(ds_name))
    with open(f'curl_{ds_name}.pkl', 'wb') as f:
        pickle.dump(curl[ds_name], f)

In [ ]:
ds = Dataset.from_generator(load_ds, gen_kwargs={'ds_name':ds_name})

In [4]:

ds = load_from_disk(ds_path)
ds

Dataset({
    features: ['text', 'url', 'en_proba', 'erased_mail', 'n_erased_mail', 'erased_phone', 'n_erased_phone', 'gopher_verdict', 'sentences', 'sentences_normalized', 'dedup_text', 'erased_dedup', 'dedup_diff'],
    num_rows: 199332
})

In [ ]:
ds = ds.add_column("id", list(range(len(ds))))
ds

In [ ]:
detect_en.erase_model()
ds=ds.map(detect_en.calculate_english, batched=True, num_proc=4)

In [ ]:
ds=ds.map(mask_pii.calculate_masked_pii, fn_kwargs={'mask_entity':'mail'}, batched=True, num_proc=4)
ds=ds.map(mask_pii.calculate_masked_pii, fn_kwargs={'mask_entity':'phone'}, batched=True, num_proc=4)
ds

In [ ]:
ds=ds.map(gopher_filter.calculate_gopher, batched=True, num_proc=4)

In [ ]:
d_sentences = ds.map(dedup.apply_normalization, batched=True, num_proc=4)

In [ ]:
dedup.calculate_sentence_count()

In [ ]:
ds = ds.map(dedup.erase_duplicates, batched=True, num_proc=4)

In [ ]:
transformers_classifiers.erase_engines()
ds = ds.map(transformers_classifiers.calculate_score, batched=True, batch_size=16, fn_kwargs={'score_label':'hate'})

In [ ]:
transformers_classifiers.erase_engines()
ds = ds.map(transformers_classifiers.calculate_score, batched=True, batch_size=16, fn_kwargs={'score_label':'nsfw'})

In [ ]:
load_nsfw_lines()
a=ds.map(check_is, batched=True,num_proc=4)
a

In [ ]:
acum=np.zeros(len(lines),dtype=int)
cases=np.zeros(len(lines),dtype=int)
for i,x in enumerate(tqdm(a)):
    is_present = np.array(x['is_present'],dtype=int)
    cases[is_present==1]=i
    acum += is_present

In [3]:
en_proba = np.array(ds['en_proba'])
en_proba

array([0.99978822, 0.99982971, 0.99979931, ..., 0.99972016, 0.99968553,
       0.99992025])

In [5]:
sum(np.array(en_proba)<0.65).item()*100/len(ds)

2.4135612947243796

In [ ]:
# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(en_proba[en_proba>0.993], bins=50, edgecolor='black')
plt.title('Histogram of en_proba')
plt.xlabel('en_proba')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
with open('english_erased.log', 'w') as f:
    for x in ds.shuffle().select(range(500)).filter(lambda x : x['en_proba']<0.65):
        pprint(x, file=f)

In [ ]:
n_erased_mail = np.array(ds['n_erased_mail'])

sum(n_erased_emails>0).item()*100/len(ds)

In [ ]:
with open('mail_masked.log', 'w') as f:
    for x in ds.shuffle().select(range(250)).filter(lambda x : x['n_erased_mail']>0):
        pprint(x, file=f)

In [ ]:
n_erased_phone = np.array(ds['n_erased_phone'])
n_erased_phone

In [ ]:
sum(n_erased_phone>0).item()*100/len(ds)

In [ ]:
with open('phone_masked.log', 'w') as f:
    for x in ds.shuffle().select(range(250)).filter(lambda x : x['n_erased_phone']>0):
        pprint(x, file=f)

In [5]:
gopher_verdict = np.array(ds['gopher_verdict'])

for err in gopher_filter.GOPHER_ERROR:
    print(f'{err}: {(gopher_verdict==err).sum().item()*100/len(ds)}')
    with open(f'gopher_{err}_erased.log', 'w') as f:
        for x in ds.shuffle().select(range(10000)).filter(lambda x : x['gopher_verdict']==err).select(range(3)):
            pprint(x, file=f)

few_words: 6.067264663977685


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

many_words: 0.0


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

short_words: 0.02608713101759878


...

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

ellipsis: 0.0702345835089198


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

few_alpha: 7.714767322858347


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
q=ds.filter(lambda batch : ['{' in x or '}' in x for x in batch['text']], batched=True)
q

In [ ]:
sentence_count_sorted = sorted(list(sentence_count.items()), key=lambda x : -x[1])
short_sentences = []
for s,c in sentence_count_sorted:
    s=s.replace('<SEP>','').replace('<PAD>','')
    if len(s.split())==1 and c>2:
        short_sentences.append((s,c))
short_sentences

In [ ]:
len(ds.filter(lambda x : len(x['dedup_text'])==0))*100/len(ds)

In [ ]:
sum(len(x) for x in tqdm(ds['dedup_text']))*100/sum(len(x) for x in tqdm(ds['text']))

In [ ]:
sum(len(x) for x in tqdm(ds['erased_dedup']))*100/sum(len(x) for x in tqdm(ds['text']))

In [ ]:
def cond(x):
    return x['en_proba']>0.65 and x['gopher_verdict']=='ok' and len(x['erased_dedup'])>0
                                                                    
with open('erased_dedup.log', 'w') as f:
    for x in ds.shuffle().select(range(250)).filter(cond):
        pprint(x, body_field='dedup_diff', file=f)

In [ ]:
# Sorting and selecting the top-20 counts
top_indices = np.argsort(acum)[-60:-20][::-1]  # Indices of top-20 counts in descending order
top_tags = [lines[i] for i in top_indices]
top_counts = acum[top_indices]*100 / len(ds)

# Plotting the histogram
plt.figure(figsize=(12, 8))
plt.bar(top_tags, top_counts, edgecolor='black')
plt.xlabel('Tags')
plt.ylabel('Counts')
plt.title('Histogram of Top-20 Tag Counts')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()